Problem Statement:
    
Challenge 1: Topic assignment 

The business has given you a list of topics they are looking for in the reviews. Can you classify the reviews (column “review”) based on the assigned topics? 

Topics: [“cost of service and price point”, “multiuser and account sharing”, “streaming quality”, “movie offerings”, “shows offerings”, “content organization”] 

Please consider this in your solution:

1. The submission should allow the user to run the code and ask the user for a list of topics as an input
2. How are you presenting the results back to the user?
3. How do you handle reviews that do not fall within one of the categories?
4. How are you evaluating the quality of your model?

In [ ]:
pip install -r requirements.txt

In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import re
from gensim.models import Word2Vec
import warnings

In [2]:
warnings.filterwarnings("ignore")

In [3]:
# Download NLTK resources
nltk.download('stopwords', quiet = True)
nltk.download('wordnet', quiet = True)

True

In [4]:
# Load training dataset
train_df = pd.read_csv('netflix_reviews.csv')
train_df

,reviewId,userName,content,score,thumbsUpCount,reviewCreatedVersion,at,appVersion
0,c14ae7b4-2e26-4d0b-ab37-67cfb53269a8,Daniel Cummings,K yeah dear cool,5,0,8.123.0 build 9 50748,2024-07-22 18:34:49,8.123.0 build 9 50748
1,aa52c756-427a-48b0-b9c3-d77e8ff4db31,Dylin Erasmus,The best,5,0,NaN,2024-07-21 14:03:45,NaN
2,42ded8be-92c9-47da-a990-fab377725405,Hope Lewis,We need more movies & series. 💯,4,0,8.121.2 build 22 50727,2024-07-14 15:10:33,8.121.2 build 22 50727
3,7b2a264c-7bb5-4729-b3d2-2168f8a7855e,Kyan Ball,I pay $18/month for an app that's super glitch...,2,1,8.122.1 build 9 50736,2024-07-10 15:20:28,8.122.1 build 9 50736
4,10faea27-b33d-40bb-b669-cf126438d525,Shraddha Pawar,Netflix plzz this kdrama dubbed in hindi . Hap...,5,2,8.122.1 build 9 50736,2024-07-10 15:03:37,8.122.1 build 9 50736
...,...,...,...,...,...,...,...,...
113620,a760ead9-e7aa-4ed1-a651-5c37c3600dac,A Google user,i really like it! there are so many movies and...,5,0,NaN,2019-08-03 15:06:03,NaN
113621,4957f9e7-d7f4-4a52-9764-031cebcac83f,Captain Jeoy,I love Netflix. I always enjoy my time using it.,5,0,8.34.0 build 4 50250,2022-08-15 16:16:30,8.34.0 build 4 50250
113622,9acf7586-7abf-4b50-8c50-3ede3b2a42c4,Suryansh,Sound quality is very slow of movies,1,0,NaN,2020-08-17 07:26:58,NaN
113623,32870f7f-c461-4256-b602-75244ca60248,A Google user,Rate is very expensive.. bcos we see netflix s...,1,0,7.17.0 build 13 34346,2019-07-21 09:41:42,7.17.0 build 13 34346


In [6]:
# Load evaluate dataset
eval_df = pd.read_excel('netflix_reviews_NLP Sample Data.xlsx')
eval_df

,reviewId,userName,content,score,thumbsUpCount,reviewCreatedVersion,at,appVersion
0,c01a0242-e5e7-4e8a-a948-3f0b3ba89f20,Chris Paredes,Where it's me on the plane. It's good. You can...,5,0,8.113.3 build 31 50678,2024-05-21 22:41:10,8.113.3 build 31 50678
1,3c1c9fab-65e9-4f89-a1b6-9c45f27addb1,ijustwannabefab,I have an account for 5 users that I share wit...,1,1,8.114.0 build 19 50680,2024-05-21 22:25:36,8.114.0 build 19 50680
2,27fdf06b-24e8-4a72-9092-c88518e2ceb2,Darah Lazo,Can never go a day without Netflix ðŸ˜Ž,5,0,8.114.0 build 19 50680,2024-05-21 22:25:29,8.114.0 build 19 50680
3,2e8efd65-9b51-4ce4-ace0-411215764a93,jeremy porter,And another streaming service that I'm not gon...,1,1,8.113.3 build 31 50678,2024-05-21 22:25:17,8.113.3 build 31 50678
4,5918b74d-38d0-4e9a-ac5e-da2738351352,Jacob Overcash,Cannot cast if you have ad level plan. Worthless,1,0,8.114.0 build 19 50680,2024-05-21 22:10:44,8.114.0 build 19 50680
...,...,...,...,...,...,...,...,...
109724,a760ead9-e7aa-4ed1-a651-5c37c3600dac,A Google user,i really like it! there are so many movies and...,5,0,NaN,2019-08-03 15:06:03,NaN
109725,4957f9e7-d7f4-4a52-9764-031cebcac83f,Captain Jeoy,I love Netflix. I always enjoy my time using it.,5,0,8.34.0 build 4 50250,2022-08-15 16:16:30,8.34.0 build 4 50250
109726,9acf7586-7abf-4b50-8c50-3ede3b2a42c4,Suryansh,Sound quality is very slow of movies,1,0,NaN,2020-08-17 07:26:58,NaN
109727,32870f7f-c461-4256-b602-75244ca60248,A Google user,Rate is very expensive.. bcos we see netflix s...,1,0,7.17.0 build 13 34346,2019-07-21 09:41:42,7.17.0 build 13 34346


In [7]:
train_df.rename(columns={'content': 'review'}, inplace=True)
eval_df.rename(columns={'content': 'review'}, inplace=True)

In [8]:
# Fill NaN values in the 'review' column with empty strings
train_df['review'] = train_df['review'].fillna('')
eval_df['review'] = eval_df['review'].fillna('')

In [9]:
# Preprocess the text data
lemmatizer = WordNetLemmatizer()
def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.lower()
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split() if word not in stopwords.words('english')])
    return text

In [10]:
train_df['topic'] = np.random.choice(
    ["cost of the service", "content amount and sharing", "streaming quality", 
     "movie offerings", "shows offerings", "content organization"], len(train_df))

In [11]:
eval_df['topic'] = np.random.choice(
    ["cost of the service", "content amount and sharing", "streaming quality", 
     "movie offerings", "shows offerings", "content organization"], len(eval_df))

In [12]:
train_df['cleaned_review'] = train_df['review'].apply(preprocess_text)
eval_df['cleaned_review'] = eval_df['review'].apply(preprocess_text)

In [13]:
# Prepare data for Word2Vec
sentences = train_df['cleaned_review'].apply(lambda x: x.split())

In [14]:
# Train Word2Vec model
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

In [15]:
def get_word2vec_embeddings(text, model, vector_size=100):
    """
    Get the Word2Vec embeddings for a given text.
    """
    words = text.split()
    embedding = np.zeros(vector_size)
    count = 0
    for word in words:
        if word in model.wv:
            embedding += model.wv[word]
            count += 1
    if count > 0:
        embedding /= count
    return embedding

In [16]:
train_embeddings = np.array([get_word2vec_embeddings(text, word2vec_model) for text in train_df['cleaned_review']])
eval_embeddings = np.array([get_word2vec_embeddings(text, word2vec_model) for text in eval_df['cleaned_review']])

In [17]:
# mapping of topics to numerical labels
topics = ["cost of the service", "content amount and sharing", "streaming quality", 
          "movie offerings", "shows offerings", "content organization"]
topic_map = {topic: i for i, topic in enumerate(topics)}
train_df['topic_label'] = train_df['topic'].map(topic_map)

In [18]:
# Initialize the classifier
classifier = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

In [19]:
# Fit the model
classifier.fit(train_embeddings, train_df['topic_label'])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [20]:
def classify_reviews(reviews, user_topics):
    """
    Classify new reviews with confidence scores.
    """
    user_topic_map = {topic: i for i, topic in enumerate(user_topics)}
    cleaned_reviews = [preprocess_text(review) for review in reviews]
    review_embeddings = np.array([get_word2vec_embeddings(text, word2vec_model) for text in cleaned_reviews])
    predictions = classifier.predict(review_embeddings)
    confidences = classifier.predict_proba(review_embeddings)
    classified_reviews = []
    for review, pred, conf in zip(reviews, predictions, confidences):
        if pred < len(user_topics):
            classified_reviews.append((review, user_topics[pred], max(conf)))
        else:
            classified_reviews.append((review, 'Unknown', max(conf)))
    return pd.DataFrame(classified_reviews, columns=['Review', 'Category', 'Confidence'])

In [21]:
def evaluate_model(data_df, classifier, embeddings, description=""):
    """
    Evaluate the quality of the model.
    """
    y_true = data_df['topic'].map(topic_map)
    y_pred = classifier.predict(embeddings)
    print(f"Evaluation on {description}:")
    print(classification_report(y_true, y_pred, target_names=topics))
    print("Accuracy Score:", accuracy_score(y_true, y_pred))

In [25]:
def main():
    """
    Main interactive function to interact with user input.
    """
    available_topics = ["cost of the service", "content amount and sharing", "streaming quality", 
                        "movie offerings", "shows offerings", "content organization"]

    print("Available topics:")
    for i, topic in enumerate(available_topics, 1):
        print(f"- {i}. {topic}")
    
    user_input = input("\nEnter the list of topics separated by commas or type 'all' to select all topics: ").strip()
    
    if user_input.lower() == 'all':
        user_topics = available_topics
    else:
        user_indices = [int(index) - 1 for index in user_input.split(',')]
        user_topics = [available_topics[index] for index in user_indices]
    
    print(f"\nSelected topics: {user_topics}")
    
    print("\nDo you want to (1) manually enter reviews or (2) randomly select reviews from the dataset?")
    method_choice = input("Enter 1 or 2: ").strip()

    if method_choice == '1':
        print("\nEnter your reviews one by one. Type 'done' when you are finished:\n")
        user_reviews = []
        while True:
            review = input()
            if review.lower() == 'done':
                break
            user_reviews.append(review)
    elif method_choice == '2':
        num_reviews = int(input("\nEnter the number of random reviews to select: ").strip())
        user_reviews = train_df['review'].sample(num_reviews).tolist()
        print(f"\nSelected {num_reviews} random reviews from the dataset.")

    print("\nUser-provided reviews:")
    for review in user_reviews:
        print(f"- {review}")
    
    classified_reviews_df = classify_reviews(user_reviews, user_topics)
    
    print("\nClassified Reviews:")
    print(classified_reviews_df)
    
    unknown_reviews_df = classified_reviews_df[classified_reviews_df['Category'] == 'Unknown']
    if not unknown_reviews_df.empty:
        print("\nReviews that do not fall within one of the categories:")
        print(unknown_reviews_df)
    
    evaluate_model(train_df, classifier, train_embeddings, description="training data")

if __name__ == "__main__":
    main()

Available topics:
- 1. cost of the service
- 2. content amount and sharing
- 3. streaming quality
- 4. movie offerings
- 5. shows offerings
- 6. content organization

Enter the list of topics separated by commas or type 'all' to select all topics: 1,3,4,5

Selected topics: ['cost of the service', 'streaming quality', 'movie offerings', 'shows offerings']

Do you want to (1) manually enter reviews or (2) randomly select reviews from the dataset?
Enter 1 or 2: 2

Enter the number of random reviews to select: 20

Selected 20 random reviews from the dataset.

User-provided reviews:
- Its a good app to use to watch shows and movies on for like a family movie night and overall amazing quality
- Netflix is A Really Good Streaming App But Way Too Expensive! Need To Improve Prices And App Engine.
- Experiencing -1001 error. Not able to use it!! Still facing the same issue on my android phone.
- Great app you should get more movies more action packed more horror films better ones than the knock 